In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_"

In [121]:
data_dir = prefix + "11_43_45"

blocksPerFile = 6
numberOfFiles = 1 # 108
firstFile = 84
lastFile = firstFile + numberOfFiles * blocksPerFile # 672

samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = samplesPerRamp * rampsPerFile

numberOfSamplesToRead = 9000000

In [122]:
timeSeries_A = np.zeros( (numberOfFiles, numberOfSamplesToRead) )
timeSeries_B = np.zeros( (numberOfFiles, numberOfSamplesToRead) )
dum = np.zeros( numberOfSamplesToRead * 2 )

fileNumber = 0

for loop in range (firstFile, lastFile, blocksPerFile ):
    print( str(loop) + ' / ' + str(lastFile - blocksPerFile) )
    # open the file containing data
    stream = data_dir + '/record' + str(loop) + '.bin'
    fd = open(stream,'rb')
    # get the data contained in the file ramp by ramp
    dum = np.fromfile(fd, dtype = np.int16, count = numberOfSamplesToRead * 2)
    timeSeries_A[fileNumber, :] = dum[ 0 : 2 * numberOfSamplesToRead : 2 ]
    timeSeries_B[fileNumber, :] = dum[ 1 : 2 * numberOfSamplesToRead : 2 ]
    
    fileNumber = fileNumber + 1
    
    fd.close()

84 / 84


In [89]:
plt.figure()
for k in range(numberOfFiles):
    plt.plot(timeSeries_A[k, :])

In [90]:
plt.figure()
for k in range(numberOfFiles):
    plt.plot(timeSeries_B[k, :])

## Rebuild the data

In [123]:
def readFile( filename, timeSerie_A, timeSerie_B ):
    fd = open(filename,'rb')
    
    dum = np.fromfile(fd, dtype = np.int16)

    timeSerie_A[:] = dum[ 0 : 2 * numberOfSamplesToRead : 2 ]
    timeSerie_B[:] = dum[ 1 : 2 * numberOfSamplesToRead : 2 ]
    
    fd.close()

### Check pulse duration in B channel data

In [124]:
timeSerie_A = np.zeros( numberOfSamplesToRead )
timeSerie_B = np.zeros( numberOfSamplesToRead )

In [125]:
nbFilesToCheck = 11
nbFilesToRebuild = 11
threshold = 6000

In [126]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    pulseDuration = np.where(timeSerie_B > threshold)[0].shape[0]
    print("file " + str(nb) + ", pulseDuration =  " + str(pulseDuration))

file 84, pulseDuration =  1000
file 90, pulseDuration =  1000
file 96, pulseDuration =  1000
file 102, pulseDuration =  1000
file 108, pulseDuration =  1000
file 114, pulseDuration =  1000
file 120, pulseDuration =  1000
file 126, pulseDuration =  1000
file 132, pulseDuration =  1000
file 138, pulseDuration =  1000
file 144, pulseDuration =  1000


### Check the triggers in the B channel data

In [127]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    triggers = np.where( np.diff(timeSerie_B) > threshold )[0]
    print("file " + str(nb) + ", threshold = " + str(triggers))

file 84, threshold = [8998690]
file 90, threshold = [8998606]
file 96, threshold = [8998533]
file 102, threshold = [8998465]
file 108, threshold = [8998383]
file 114, threshold = [8998290]
file 120, threshold = [8998216]
file 126, threshold = [8998130]
file 132, threshold = [8998059]
file 138, threshold = [8997974]
file 144, threshold = [8997929]


### Rebuild the data files

In [128]:
timeSerie_C = np.zeros( samplesPerFile )
timeSerie_D = np.zeros( samplesPerFile )

In [129]:
rebuilt_A = np.zeros((nbFilesToRebuild, samplesPerFile))
rebuilt_B = np.zeros((nbFilesToRebuild, samplesPerFile))
for k in range(11):
    nb        = str( int( k     * blocksPerFile + firstFile) )
    nb_plus_1 = str( int( (k+1) * blocksPerFile + firstFile) )
    readFile( data_dir + "/record" + nb        +".bin", timeSerie_A, timeSerie_B )
    readFile( data_dir + "/record" + nb_plus_1 +".bin", timeSerie_C, timeSerie_D )
    trigs = np.where( np.diff(timeSerie_B) > threshold )[0]
    trig = trigs[0] + 1

    rebuilt_B[ k, 0 : samplesPerFile - trig ] = timeSerie_B[ trig : ]
    rebuilt_B[ k, samplesPerFile - trig : ] = timeSerie_D[ 0 : trig ]
    
    rebuilt_A[ k, 0 : samplesPerFile - trig ] = timeSerie_A[ trig : ]
    rebuilt_A[ k, samplesPerFile - trig : ] = timeSerie_C[ 0 : trig ]
    
    print("file " + str(nb) + ", trigs = " + str(trigs))

file 84, trigs = [8998690]
file 90, trigs = [8998606]
file 96, trigs = [8998533]
file 102, trigs = [8998465]
file 108, trigs = [8998383]
file 114, trigs = [8998290]
file 120, trigs = [8998216]
file 126, trigs = [8998130]
file 132, trigs = [8998059]
file 138, trigs = [8997974]
file 144, trigs = [8997929]


In [130]:
for k in range(10):
    plt.plot(rebuilt_A[k,0:6000])

In [131]:
A_reshaped = rebuilt_A.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)
B_reshaped = rebuilt_B.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)

In [132]:
coupling_A = np.average(A_reshaped, 0)
A_reshaped -= coupling_A

In [133]:
for k in range(10):
    plt.plot(A_reshaped[k,:])

In [134]:
ifft_a = np.fft.ifft( (A_reshaped[:,0:3000]), axis = 1 )

In [136]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,0:500] ) ), aspect='auto' )

In [ ]:
plt.plot((A_reshaped - coupling_A)[:,0:3000])

In [137]:
rd_a = np.fft.fft( ifft_a[:,0:500], axis = 0 )

In [139]:
plt.matshow( 20 * np.log10( np.abs( rd_a ) ), aspect='auto' )

In [31]:
22500/3000

7.5

In [140]:
np.save('A_reshaped.npy', A_reshaped)